In [ ]:
!rm ~/.fiftyone -rf

In [ ]:
import os
from pathlib import Path

import fiftyone as fo
import hydra

from lightning_pose.utils import pretty_print_str
from lightning_pose.utils.fiftyone import (
    FiftyOneFactory,
    FiftyOneImagePlotter,
    FiftyOneKeypointVideoPlotter,
    check_dataset,
)

In [ ]:
project_dir = Path.cwd().parent

In [ ]:
data_dir = project_dir / "data/mirror-mouse-example"

In [ ]:
# folders in 'outputs' containing results to analyse
outputs = [
    '2023-11-20/03-47-21',
    '2023-11-20/02-21-30',
    '2023-11-20/02-53-36',
    '2023-11-17/04-40-21'
]

# append outputs directory to only manipulate absolute paths
output_dir = project_dir / "outputs"
outputs = [str(output_dir / path) for path in outputs]

In [ ]:
with hydra.initialize(version_base=None, config_path="../config"):
    cfg = hydra.compose(
        config_name="config_mirror-mouse-example",
        overrides=[
            f"data.data_dir={data_dir}",
            f"eval.hydra_paths=[{','.join(outputs)}]",
        ]
    )

In [ ]:
pretty_print_str(
    f"Launching a job that creates {cfg.eval.fiftyone.dataset_to_create} "
    "FiftyOne.Dataset"
)

FiftyOneClass = FiftyOneFactory(dataset_to_create=cfg.eval.fiftyone.dataset_to_create)()

# initializes everything
fo_plotting_instance = FiftyOneClass(cfg=cfg)

# internally loops over models
dataset = fo_plotting_instance.create_dataset()

# create metadata and print if there are problems
check_dataset(dataset)

# print the name of the dataset
fo_plotting_instance.dataset_info_print()

In [ ]:
username = os.environ['USER']
fo.app_config.proxy_url = (
    f"https://jupyter.nesi.org.nz/user/{username}/proxy/{cfg.eval.fiftyone.port}/"
)
fo.launch_app(dataset, port=cfg.eval.fiftyone.port)